In [1]:
import mlflow
import ipywidgets as widgets
from ipywidgets import interact
import os
import glob
import torch
from pprint import pprint
from easydict import EasyDict

import numpy as np
import torch

In [2]:
MLFLOW_TRACKING_URI = "/app/mlruns/"
EXPERIMENT_NAME = "Synthetic data"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [3]:
EXPERIMENT_ID = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
df = mlflow.search_runs(experiment_ids=[EXPERIMENT_ID])

___

### Select runs with $T=1$

In [4]:
row_index = (df["params.dataset_n_timeframes"] == "1")

# Select runs with performance better than random
row_index &= (df["metrics.test_rec_ratio_to_pop_mean"] < 1)

In [5]:
# columns
normalized_metrics = df.columns[df.columns.str.startswith("metrics.test_") & df.columns.str.contains("ratio")]

dataset_params = df.columns[df.columns.str.startswith("params.dataset_")].to_list()

params = df.columns[df.columns.str.startswith("params.")].to_list()

arch_params = [
    'params.n_channels_enc', 
    'params.n_channels_dec_c', 
    'params.n_channels_dec_s', 
    'params.latent_dim_c', 
    'params.latent_dim_s',
    'params.z_aggr_function',
    'params.reduction_factors',
]

# To not display columns that have the same value for all rows
# https://stackoverflow.com/questions/57365283/how-to-show-columns-that-have-different-values-in-rows
def diff_cols(df):
    my_cols = []
    for col in df.columns:
        if df[col].nunique(dropna=False) > 1:
            my_cols.append(col)
    return df[my_cols].copy()

In [6]:
columns = ["run_id"] + params + normalized_metrics.tolist()

diff_cols(
    df[columns][row_index].reset_index(drop=True)
)

,run_id,params.patience,params.latent_dim_s,params.epochs,params.latent_dim_c,params.lr,params.platform,metrics.test_rec_ratio_to_pop_mean_c,metrics.test_rec_ratio_to_time_mean,metrics.test_rec_ratio_to_pop_mean
0,e43d2ee35025439d99999ca384c17380,20,32,2000,16,0.0003,f6dd0d45d567,0.016688,1.074090,0.154649
1,113ac3571ed9449f8c228a107aa0eb00,20,32,2000,16,0.0003,f6dd0d45d567,0.014107,1.065081,0.137841
2,97f869d0d5004886842fc63699df8ac0,20,32,2000,16,0.003,f6dd0d45d567,0.036501,3.403358,0.583607
3,0def8b8729754ae49aca6e868e9ecd2c,10,16,1000,16,0.003,f6dd0d45d567,0.036892,3.983048,0.673120
4,10a359ba5f79444d9bd641c77c431ef2,10,16,1000,16,0.0003,b5cc16a9c258,0.017136,1.252234,0.198890
5,9491c5fa1b014fceb48fabea0385e8d5,10,8,1000,8,0.0003,b5cc16a9c258,0.025837,2.270566,0.348402


In [7]:
good_runs = df[row_index].run_id.values
run_w = widgets.Select(options=good_runs)

In [8]:
@interact
def get_ckptpath(run=run_w):
    
    # chkpt_dir = f"{MLFLOW_TRACKING_URI}/{EXPERIMENT_ID}/{RUNID}/"
    
    REPO_DIR = "/app/Rodrigo_repos/CardiacMotion"
    global ckpt_path, model_weights    
    ckpt_path = glob.glob(f"{REPO_DIR}/1/{run}/checkpoints/*ckpt")    
    if len(ckpt_path) == 1:
      ckpt_path = ckpt_path[0]
    elif len(ckpt_path) == 0:
      ckpt_path = None
    
    model_weights = torch.load(ckpt_path)["state_dict"]
    model_weights = EasyDict(model_weights)
    
    print(mlflow.get_run(run).data.metrics["test_rec_ratio_to_pop_mean"])
    return ckpt_path

interactive(children=(Select(description='run', options=('e43d2ee35025439d99999ca384c17380', '113ac3571ed9449f…

In [9]:
n_params = 0

for module_name, weights in model_weights.items():
    module_name = module_name.replace("model.", "")
    n_params += np.prod(weights.shape)
    # print(f'{module_name}: {weights.shape}')
    
n_params

633840